전처리 및 모델링

In [1]:
import os 
import random
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [2]:
train_org = pd.read_csv('./open/open/train.csv') 
test_org = pd.read_csv('./open/open/test.csv')
ss = pd.read_csv('./open/open/sample_submission.csv')

display(train_org.head())
display(test_org.head())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,ACCIDENT_39612,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,ACCIDENT_39613,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차


In [3]:
train_df = train_org.copy()
test_df = test_org.copy()


In [4]:
train_df.head()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39609 non-null  object
 1   사고일시         39609 non-null  object
 2   요일           39609 non-null  object
 3   기상상태         39609 non-null  object
 4   시군구          39609 non-null  object
 5   도로형태         39609 non-null  object
 6   노면상태         39609 non-null  object
 7   사고유형         39609 non-null  object
 8   사고유형 - 세부분류  39609 non-null  object
 9   법규위반         39609 non-null  object
 10  가해운전자 차종     39609 non-null  object
 11  가해운전자 성별     39609 non-null  object
 12  가해운전자 연령     39609 non-null  object
 13  가해운전자 상해정도   39609 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         39609 non-null  int64 
 19  중상자수         39609 non-nu

In [5]:
columns = train_df.columns
columns

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형',
       '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',
       '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수',
       '경상자수', '부상자수', 'ECLO'],
      dtype='object')

In [6]:
def making_val_table(df):
    '''입력받은 데이터셋의 유효성 검증을 위한 요약 테이블'''
    # dtypes 
    df_dtypes = df.dtypes
    
    # nunique
    df_nunique = df.nunique()
    
    # null_values
    df_nan = df.isna().sum()
    
    val_table = pd.concat([df_dtypes, df_nunique, df_nan], axis=1)
    val_table.columns = ['dtype', 'nunique', 'nan']
        
    return val_table.reset_index()

In [7]:
# 변수지정 및 동기화
y_train = train_df['ECLO']

cols = test_df.columns
X_train = train_df[cols]
X_test = test_df

In [8]:
X_train.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차


In [9]:
#변환 및 컬럼생성, 삭제
def feat_eng(df):
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    
    df['월'] = df['사고일시'].dt.month
    df['일'] = df['사고일시'].dt.day
    df['시'] = df['사고일시'].dt.hour

    subs = ['ID','사고일시','기상상태','시군구']
    df = df.drop(subs, axis=1)
    

    df = pd.get_dummies(df)

    return df

In [10]:
X_train_eng = feat_eng(X_train)
X_test_eng = feat_eng(X_test)

In [11]:
X_train_eng.info()
X_test_eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   월                     39609 non-null  int32
 1   일                     39609 non-null  int32
 2   시                     39609 non-null  int32
 3   요일_금요일                39609 non-null  bool 
 4   요일_목요일                39609 non-null  bool 
 5   요일_수요일                39609 non-null  bool 
 6   요일_월요일                39609 non-null  bool 
 7   요일_일요일                39609 non-null  bool 
 8   요일_토요일                39609 non-null  bool 
 9   요일_화요일                39609 non-null  bool 
 10  도로형태_교차로 - 교차로부근      39609 non-null  bool 
 11  도로형태_교차로 - 교차로안       39609 non-null  bool 
 12  도로형태_교차로 - 교차로횡단보도내   39609 non-null  bool 
 13  도로형태_기타 - 기타          39609 non-null  bool 
 14  도로형태_단일로 - 고가도로위      39609 non-null  bool 
 15  도로형태_단일로 - 교량         39609 non-null  bool 
 16  도로형태

In [12]:
X_train_eng.head()

,월,일,시,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,...,도로형태_주차장 - 주차장,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,사고유형_차대사람,사고유형_차대차,사고유형_차량단독
0,1,1,0,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,True,False,False
1,1,1,0,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,True,False,False
2,1,1,1,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,True,False,False
3,1,1,2,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
4,1,1,4,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False


In [13]:
X_test_eng.head()

,월,일,시,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,...,도로형태_주차장 - 주차장,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,사고유형_차대사람,사고유형_차대차,사고유형_차량단독
0,1,1,1,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,True,False,False
1,1,1,1,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,True,False,False
2,1,1,4,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False
3,1,1,4,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False
4,1,1,6,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False


eda과정중에 평균값을 값을 내면 유의미한 차이가 없을 것이라고 생각하여 평균값 보다는 원핫인코딩이 예측률이 더 높고 합당하다고 생각하였습니다. 떄문에 기상상태, 시군구를 제거하였습니다.

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_eng, y_train , test_size=0.2, random_state=42)

In [15]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)
y_val_pred = rf.predict(X_val)

In [16]:
y_pred = rf.predict(X_test_eng)
y_pred

array([4.16      , 2.95      , 7.19      , ..., 5.73      , 7.48      ,
       6.29466667])

In [17]:
y_pred = rf.predict(X_test_eng)
y_pred

array([4.16      , 2.95      , 7.19      , ..., 5.73      , 7.48      ,
       6.29466667])

In [21]:
ss['ECLO'] = y_pred
ss.head()

,ID,ECLO
0,ACCIDENT_39609,4.160000
1,ACCIDENT_39610,2.950000
2,ACCIDENT_39611,7.190000
3,ACCIDENT_39612,7.170000
4,ACCIDENT_39613,4.946667


In [22]:
OUTPUT_PATH = './output/'
ss.to_csv(f'./open/open/submission.csv', index=False)